In [1]:
import geopandas as gpd

In [2]:
driving = gpd.read_file('../data/googleQueryDriving/googleQueryDriving.shp')
transit = gpd.read_file('../data/googleQueryTransit/googleQuery.shp')

In [3]:
driving.head()

,CO_FRAC_RA,geometry,tiempos,trafico
0,1_1_1,POLYGON ((-58.37189352634645 -34.5919935640870...,641,969.0
1,1_12_1,POLYGON ((-58.38592548750878 -34.6044398706025...,631,734.0
2,1_12_10,POLYGON ((-58.37878669239328 -34.6052849003551...,501,582.0
3,1_12_11,POLYGON ((-58.36732497447693 -34.6056209207810...,37,36.0
4,1_12_2,POLYGON ((-58.38453585809192 -34.6043583533245...,449,558.0


In [4]:
transit.head()

,CO_FRAC_RA,dinero,geometry,tiempos
0,1_1_1,6.00,POLYGON ((-58.37189352634645 -34.5919935640870...,1601
1,1_12_1,6.25,POLYGON ((-58.38592548750878 -34.6044398706025...,751
2,1_12_10,NaN,POLYGON ((-58.37878669239328 -34.6052849003551...,274
3,1_12_11,NaN,POLYGON ((-58.36732497447693 -34.6056209207810...,97
4,1_12_2,7.50,POLYGON ((-58.38453585809192 -34.6043583533245...,591


In [14]:
data = gpd.read_file('../data/radiosQuearyCABA/radiosQuearyCABA.shp')
data.drop(['mejorMod','x','y'],axis=1,inplace=True)
data.head()

,CO_FRAC_RA,geometry,tiemposAut,tiemposBic,tiemposTra
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",10.0,15.0,14.0
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",10.0,12.0,8.0
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",6.0,4.0,8.0
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",2.0,1.0,2.0
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",7.0,9.0,8.0


In [15]:
data['transitGoog'] = transit.tiempos*1./60 
data['driveGoog'] = driving.tiempos*1./60 
data['traficGoog'] = driving.trafico*1./60 
data.head()

,CO_FRAC_RA,geometry,tiemposAut,tiemposBic,tiemposTra,transitGoog,driveGoog,traficGoog
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",10.0,15.0,14.0,26.683333,10.683333,16.150000
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",10.0,12.0,8.0,12.516667,10.516667,12.233333
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",6.0,4.0,8.0,4.566667,8.350000,9.700000
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",2.0,1.0,2.0,1.616667,0.616667,0.600000
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",7.0,9.0,8.0,9.850000,7.483333,9.300000


In [16]:
data['driveTot'] = data.traficGoog + data.driveGoog
data.head()

,CO_FRAC_RA,geometry,tiemposAut,tiemposBic,tiemposTra,transitGoog,driveGoog,traficGoog,driveTot
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",10.0,15.0,14.0,26.683333,10.683333,16.150000,26.833333
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",10.0,12.0,8.0,12.516667,10.516667,12.233333,22.750000
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",6.0,4.0,8.0,4.566667,8.350000,9.700000,18.050000
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",2.0,1.0,2.0,1.616667,0.616667,0.600000,1.216667
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",7.0,9.0,8.0,9.850000,7.483333,9.300000,16.783333


In [17]:
data['traficPerc'] = data.traficGoog/data.driveTot

In [19]:
data['bikePerc'] = data.tiemposBic / data.driveTot

In [20]:
data.to_file('../data/basefinal/')